In [1]:
# !/usr/bin/python
# -*- coding: UTF-8 -*-
# 自动打卡
# 创建session逐页访问，获取带参数的完整url（否则无法通过服务器校检）以及隐藏域

import requests
import parsel
import re

In [2]:
login_name = '123456'
login_password = '123456'

# 二进制为两图片，待完善
your_form = {
    'ctl00$cph_right$e_location': 'xx省xx市xx区xx镇',
    'ctl00$cph_right$e_health$0': '无不适',
    'ctl00$cph_right$e_temp': '36.3',
    'ctl00$cph_right$e_travel:': 'xx',
    'ctl00$cph_right$e_annex': '(二进制)',
    'ctl00$cph_right$e_annex2': '(二进制)',
    'ctl00$cph_right$e_describe': '',
}

headers = {
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
  'Accept-Encoding': 'gzip, deflate, br',
  'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,zh-TW;q=0.7,ja;q=0.6,vi;q=0.5',
  'Cache-Control': 'max-age=0',
  'Connection': 'keep-alive',
  'Host': 'ssp.xxxx.edu.cn',
  'Origin': 'https://ssp.xxxx.edu.cn',
  'Referer': 'https://ssp.xxxx.edu.cn/login.aspx',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-User': '?1',
  'Upgrade-Insecure-Requests': '1',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
}

In [3]:
import time

s = requests.Session()

# 访问登陆页面，获得session
url_login = "https://ssp.xxxx.edu.cn/login.aspx"
r = s.get(url_login)
# 获取cookie
cookies = requests.cookies.RequestsCookieJar()
s.cookies.update(cookies)

# 获取三个隐藏域
selector = parsel.Selector(r.text)
hidden1 = selector.css("#__VIEWSTATE ::attr(value)").getall()[0]
hidden2 = selector.css("#__VIEWSTATEGENERATOR ::attr(value)").getall()[0]
hidden3 = selector.css("#__EVENTVALIDATION ::attr(value)").getall()[0]

# 登录
payload = {
    '__EVENTTARGET': 'logon',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': hidden1,
    'log_username': login_name,
    'log_password': login_password,
    '__VIEWSTATEGENERATOR': hidden2,
    '__EVENTVALIDATION': hidden3
}
response = s.post(url_login, headers=headers, data=payload)

with open("log.txt", 'a+', encoding='utf-8') as f:
    try:
        # 获取疫情防控页面地址
        global url_prevention
        url_prevention = re.search(r'<a href="(.+)">疫情防控', response.text).group(1).strip("\\")
        url_prevention = "https://ssp.xxxx.edu.cn/{}".format(url_prevention)
        f.write("INFO: Page requested succeed | {}\n".format(time.asctime(time.localtime(time.time()))))
    except Exception as e:
        f.write("INFO: Page failed, check your login info | {}\n".format(time.asctime(time.localtime(time.time()))))
        raise e

In [5]:
# 进入疫情防控页面
payload = {}

response = s.request("GET", url_prevention, headers=headers, data=payload)

with open("log.txt", 'a+', encoding='utf-8') as f:
    try:
        global url_clock
        # 获取健康打卡页面地址
        url_clock = re.search(r'<a href="(.+)">健康打卡', response.text).group(1).strip("\\")
        url_clock = "https://ssp.xxxx.edu.cn/{}".format(url_clock)
        
        # 获取三个隐藏域
        selector = parsel.Selector(response.text)
        hidden1 = selector.css("#__VIEWSTATE ::attr(value)").getall()[0]
        hidden2 = selector.css("#__VIEWSTATEGENERATOR ::attr(value)").getall()[0]
        hidden3 = selector.css("#__EVENTVALIDATION ::attr(value)").getall()[0]
        f.write("  INFO: url_prevention succeed\n")
    except Exception as e:
        f.write("  INFO: url_prevention failed | {}\n".format(time.asctime(time.localtime(time.time()))))
        raise e

In [7]:
# 进入健康打卡页面
# payload = {
#     '__EVENTTARGET': '',
#     '__EVENTARGUMENT': '',
#     '__LASTFOCUS': '',
#     '__VIEWSTATE': hidden1,
#     '__VIEWSTATEGENERATOR': hidden2,
#     '__EVENTVALIDATION': hidden3,
# }

# response = s.request("GET", url_clock, headers=headers, data=payload)
# 1.25 updated: 变成get请求了
response = s.request("GET", url_clock, headers=headers)

with open("log.txt", 'a+', encoding='utf-8') as f:
    try:
        # 进入开始填报页面
        global url_clock_form
        url_clock_form = url_clock
        
        # 获取三个隐藏域
        selector = parsel.Selector(response.text)
        hidden1 = selector.css("#__VIEWSTATE ::attr(value)").getall()[0]
        hidden2 = selector.css("#__VIEWSTATEGENERATOR ::attr(value)").getall()[0]
        hidden3 = selector.css("#__EVENTVALIDATION ::attr(value)").getall()[0]
        
        f.write("  INFO: url_clock succeed\n")
    except Exception as e:
        f.write("  INFO: url_clock failed | {}\n".format(time.asctime(time.localtime(time.time()))))
        raise e
# print(s.cookies.get_dict())

In [9]:
# 1.25 updated: 网站取消了这个页面

# # 进入开始填报页面
# payload = {'__EVENTTARGET': '',
# '__EVENTARGUMENT': '',
# '__VIEWSTATE': hidden1, 
# 'ctl00$cph_right$e_ok': 'on',
# 'ctl00$cph_right$ok_submit': '开始填报',
# '__VIEWSTATEGENERATOR': hidden2,
# '__EVENTVALIDATION': hidden3,
# }

# response = s.request("POST", url_clock_form, headers=headers, data=payload)

# with open("log.txt", 'a+', encoding='utf-8') as f:
#     try:
#         # 获取提交地址
#         global url_handle
#         url_handle = url_clock_form
        
#         # 获取三个隐藏域
#         selector = parsel.Selector(response.text)
#         hidden1 = selector.css("#__VIEWSTATE ::attr(value)").getall()[0]
#         hidden2 = selector.css("#__VIEWSTATEGENERATOR ::attr(value)").getall()[0]
#         hidden3 = selector.css("#__EVENTVALIDATION ::attr(value)").getall()[0]
        
#         f.write("  INFO: url_clock_form succeed\n")
#     except Exception as e:
#         f.write("  INFO: url_clock_form failed | {}\n".format(time.asctime(time.localtime(time.time()))))
#         raise e

In [16]:
# 提交
payload = {
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__VIEWSTATE': hidden1, 
    '__VIEWSTATEGENERATOR': hidden2,
    '__EVENTVALIDATION': hidden3,
    'ctl00$cph_right$e_submit': '提交保存'
}

for k, v in your_form.items():
    payload[k] = v

payload['__VIEWSTATE'] = payload['__VIEWSTATE']

response = s.request("POST", url_clock_form, headers=headers, data=payload)

with open("log.txt", 'a+', encoding='utf-8') as f:
    if response.status_code == 200:
        f.write("  INFO: url_handle succeed\n")
    else:
        f.write("  INFO: url_handle failed, code: {} | {}\n".format(response.status_code, time.asctime(time.localtime(time.time()))))

response

<Response [200]>